# Knative

Knative (ausgesprochen kay-nay-tiv) erweitert Kubernetes um eine Reihe von Middleware-Komponenten, die für die Erstellung moderner, container-basierter Anwendungen unerlässlich sind.

Knative Komponenten konzentrieren sich auf die Lösung alltäglicher, aber schwieriger Aufgaben wie:

* Bereitstellen eines Containers
* Routing und Verwaltung des Datenverkehrs mit blue / green Bereitstellung
* Automatische Skalierung des Workloads
* Bindung laufender Dienste an mehrere Ökosysteme

***
Eventing 
--------

![](https://user-images.githubusercontent.com/16281246/116248768-1fe56080-a73a-11eb-9a85-8bdccb82d16c.png)

Quelle: [knative.dev](https://knative.dev/docs/getting-started/first-broker/)
- - -

Mit Knative Eventing stehen zusammensetzbare Grundelemente bereit, um ein System von Ereignisquellen (Producer) und Ereigniskonsumenten (Consumer) zu ermöglichen.

Dabei sind folgende Funktionalitäten möglich:
* Ich möchte nur Events veröffentlichen, es ist mir egal, wer sie konsumiert.
* Ich möchte nur Events wie X konsumieren, es ist mir egal, wie sie veröffentlicht werden.
* Ich möchte Events durch eine Reihe von Schritten transformieren.

### Installation

Zuerst brauchen wir einen [Channel Based Broker](https://knative.dev/v0.13-docs/eventing/channel-based-broker/) und den dazugehörenden [In Memory Channel](https://knative.dev/docs/eventing/channels/channels-crds/).

In [ ]:
%%bash
kubectl apply -f https://github.com/knative/eventing/releases/download/knative-v1.4.1/eventing-crds.yaml
kubectl apply -f https://github.com/knative/eventing/releases/download/knative-v1.4.1/eventing-core.yaml
kubectl apply -f https://github.com/knative/eventing/releases/download/knative-v1.4.1/in-memory-channel.yaml

Überprüfen der Installation. Es müssen 4 Container gestartet sein 

In [ ]:
%%bash
kubectl get pods -n knative-eventing

## Applikation

In [ ]:
%%bash
kubectl create namespace event-example
kubectl label namespace event-example eventing.knative.dev/injection=enabled
cat <<%EOF% | kubectl apply -n event-example -f -
apiVersion: eventing.knative.dev/v1
kind: Broker
metadata:
 name: default
 namespace: event-example
%EOF%

In [ ]:
%%bash
kubectl --namespace event-example get Broker default

### Creating event consumers

Als Event Consumer starten wir unseren Service `hello-display` als Deployments.

Der Sourcecode zum Services ist auf https://github.com/knative/eventing-contrib/tree/master/cmd/event_display zu finden.

In [ ]:
%%bash
kubectl --namespace event-example apply --filename - << END
apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello-display
spec:
  replicas: 1
  selector:
    matchLabels: &labels
      app: hello-display
  template:
    metadata:
      labels: *labels
    spec:
      containers:
        - name: event-display
          # Source code: https://github.com/knative/eventing-contrib/tree/master/cmd/event_display
          image: gcr.io/knative-releases/knative.dev/eventing-contrib/cmd/event_display

---

# Service pointing at the previous Deployment. This will be the target for event
# consumption.
  kind: Service
  apiVersion: v1
  metadata:
    name: hello-display
  spec:
    selector:
      app: hello-display
    ports:
    - protocol: TCP
      port: 80
      targetPort: 8080
END

In [ ]:
! kubectl --namespace event-example get all

### Creating Triggers

Trigger verbinden die Events mit unseren Service "hello-display".

In [ ]:
%%bash
kubectl --namespace event-example apply --filename - << END
apiVersion: eventing.knative.dev/v1
kind: Trigger
metadata:
  name: hello-display
  namespace: event-example
spec:
  broker: default
  filter:
    attributes:
      type: greeting
  subscriber:
    ref:
     apiVersion: v1
     kind: Service
     name: hello-display
END

In [ ]:
! kubectl --namespace event-example get triggers

Sobald der Trigger READY ist, können wir Events gegen diesen Feuern:

In [ ]:
%%bash
curl -v "http://broker-ingress.knative-eventing/event-example/default" \
-X POST \
-H "Ce-Id: say-hello" \
-H "Ce-Specversion: 1.0" \
-H "Ce-Type: greeting" \
-H "Ce-Source: not-sendoff" \
-H "Content-Type: application/json" \
-d '{"msg":"Hello Knative!"}'
 

Als Prüfung ob der Event auch angekommen ist, geben wird das Log unseres Consumers aus:

In [ ]:
! kubectl --namespace event-example logs -l app=hello-display --tail=100

Aufräumen

In [ ]:
%%bash
kubectl delete namespace event-example
kubectl delete namespace knative-eventing
